In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [5]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [6]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [18]:
ReflectiveDLL_ProcessInjection = spark.sql(
'''
SELECT 
    b.process_path,
    b.process_target_name,
    b.process_target_id,
    b.thread_new_id,
    a.process_id,
    a.process_granted_access
FROM sysmon_events b
INNER JOIN(
SELECT event_id, process_granted_access, process_guid, process_id
FROM sysmon_events 
WHERE event_id = 10
AND (process_granted_access & 5178) == 5178) -- 5178 is decimal for 0x143A. The minimal privileges you need to access process handle
) a
ON a.process_guid = b.process_guid
WHERE b.event_id = 8
'''
).show(1,False)


+---------------------------------------------------------+-------------------+-----------------+-------------+----------+----------------------+
|process_path                                             |process_target_name|process_target_id|thread_new_id|process_id|process_granted_access|
+---------------------------------------------------------+-------------------+-----------------+-------------+----------+----------------------+
|c:\windows\system32\windowspowershell\v1.0\powershell.exe|notepad.exe        |7924             |9180         |5452      |2047999               |
+---------------------------------------------------------+-------------------+-----------------+-------------+----------+----------------------+
only showing top 1 row

